<a href="https://colab.research.google.com/github/liangli217/LLM_learning/blob/main/Agents_with_OpenAI_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://drive.google.com/uc?export=view&id=1oCldVicFepwdYbPHcD8Ui4VeyQVJYgut" height="400"><center>

In [2]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 15.2 MB/s eta 0:00:00


In [39]:
from openai import OpenAI
from typing import List, Optional, Dict, Any
from dataclasses import dataclass
from agents import Agent, Runner, function_tool
import time, random, json

In [5]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
import os
os.environ['OPENAI_API_KEY'] = api_key

#Proper client initialization
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [14]:
model = "gpt-4o"
user_prompt ="What's the best performing ETF?"

In [22]:
class BaseLLM :

  def __init__(self, model ):
    self.model = model

  def generate(self, user_prompt:str, max_tokens = 1000):

    print("BaseLLM: making single API call...")
    start_time = time.time()
    response = client.completions.create(
        model=self.model,
        max_tokens=max_tokens,
        input =[{"role": "user", "content": user_prompt}]
    )
    end_time = time.time()

    return {
        "content": response.output_text,
        "tokens_used": response.usage.total_tokens,
        "response_time": end_time - start_time
    }


In [23]:
base_llm = BaseLLM(model)
llm_result = base_llm.generate(user_prompt)

BaseLLM: making single API call...


In [27]:
print("\n" + "="*50)
print("BASE LLM:",model)
print("="*50)
print(f"Response:")
print(llm_result['content'])
print(f"Tokens: {llm_result['tokens_used']}, Time: {llm_result['response_time']:.2f}s")


BASE LLM: gpt-4o
Response:
Determining the "best" performing ETF can depend on the specific time frame, market conditions, and what you're looking to achieve. Some ETFs may be top performers in the short term but may not be suitable for long-term investments. Common performance metrics include total return over specific periods or performance compared to a benchmark index.

To find the current best-performing ETF, you would typically look at financial news websites or platforms like Morningstar, Bloomberg, or Yahoo Finance. They often have updated lists and reports on ETFs that are currently outperforming others. Remember, past performance doesn't guarantee future results, and it's important to consider factors like fees, investment strategy, and risk when evaluating an ETF.
Tokens: 149, Time: 6.94s


In [34]:
model = "gpt-5"
user_prompt ="What's the best performing ETF (US only), provide top 3 for the best performance in the YTD?"

In [50]:
base_llm = BaseLLM(model)
llm_result = base_llm.generate(user_prompt)

BaseLLM: making single API call...


In [51]:
print("\n" + "="*50)
print("BASE LLM:",model)
print("="*50)
print(f"Response:")
print(llm_result['content'])
print(f"Tokens: {llm_result['tokens_used']}, Time: {llm_result['response_time']:.2f}s")


BASE LLM: gpt-4o
Response:
Buying a home in the U.S. is a multi-step process that typically involves the following stages:

1. **Financial Preparation:**
   - **Credit Score:** Check and improve your credit score if necessary.
   - **Budget:** Determine how much you can afford, including down payment and monthly mortgage payments.
   - **Pre-Approval:** Get pre-approved for a mortgage to understand your price range and prove to sellers you're serious.

2. **Finding a Home:**
   - **Real Estate Agent:** Hire a qualified real estate agent to guide you.
   - **Home Search:** Identify your needs and wants in a home and start searching.
   - **Visit Homes:** Tour potential homes to assess their condition and suitability.

3. **Making an Offer:**
   - **Offer Price:** Decide on the price based on market conditions and the home's value.
   - **Negotiation:** Negotiate terms with the seller, which may include contingencies like inspections.

4. **Due Diligence:**
   - **Inspections:** Conduct

**Reasoning LLM**

In [44]:
class ReasoningLLM:

  def __init__(self, base_llm: BaseLLM):
    self.base_llm = base_llm
    self.conversation_history = []

  def analyze_query(self, query: str):

    analysis_prompt = f"""Analyze this query and determin the best approach to answer it comprehensively:
    Query: "{query}"

    Respond with:
    1. Query type(factual, process comparion, etc.)
    2. Key components to address
    3. Suggested breakdown steps

    Keep resposne concise and structured


    """

    print("Reasoning LLM: Step 1 - Analzying query..")
    response = self.base_llm.generate(analysis_prompt)

    return {"analysis": response["content"], "tokens": response["tokens_used"]}



  def decompose_problem(self, query: str, analysis: str):
    decomposition_prompt =f"""
    Based on this analysis: {analysis}

    Break down the query "{query}" into 3-4 specific sub-question that, when answered together,
    will provide a comprehensive response. List only the questions, one per line.
    """

    print("Reasoning LLM: Step 2- Decomposing problem...")
    response = self.base_llm.generate(decomposition_prompt)

    questions = [q.strip() for q in response['content'].split('\n') if q.strip() and '?' in q]
    return questions[:4] # Limit to 4 questionsj


  def answer_sub_questions(self, question: str) -> str:

    focused_prompt = f"""
    Answer this specific question about selecting top 3 ETF with precise, technical details:
    {question}

    Provide a clear, factual answer focusing only on this aspect.
    """

    print(f"ReasoiningLLM: Step 3 - Answering: {question[:50]}...")
    response = self.base_llm.generate(focused_prompt)
    return response["content"]


  def verify_response(self, question: str, answer: str) -> Dict[str, Any]:
    verification_prompt = f"""

    Question: {question}
    Answer: {answer}

    Evaluate this answer on a scale of 1-10 for:
    1. Accuracy
    2. Completeness
    3. Clarity

    Respond with just three numbers and any critical missing information

    """

    print("ReasoningLLM: Step 4- Verifying response...")
    response = self.base_llm.generate(verification_prompt)
    return {"verification": response["content"]}


  def synthesie_final_response(self, query: str, qa_pairs: List[tuple]) -> str:
      synthesis_prompt =f"""
      Original question: {query}

      Sub-questions and answers with verifications:

        {chr(10).join([
            f"Q: {qa['question']}\nA: {qa['answer']}\nVerification: {qa['verification']}\n"
            for qa in qa_pairs
        ])}

        Synthesize these into a well-structured, comprehensive answer to the original questions.
        Use clear steps/sections and ensure logical flow.
        """

      print("ReasoningLLM: Ste 5 - Synthesizing final response...")
      response = self.base_llm.generate(synthesis_prompt)
      return response["content"]



  def reason(self, query: str) -> str:
    print(f"\n ReasoningLLM: Processing '{query}")
    start_time = time.time()

    # Step 1: analyze query
    analysis = self.analyze_query(query)

    # Step 2: decompose into sub-questions
    sub_questions = self.decompose_problem(query, analysis["analysis"])

    # Step 3&4: Answer and verify each sub-question
    qa_pairs = []

    for question in sub_questions:
      answer = self.answer_sub_questions(question)
      verification = self.verify_response(question, answer)
      qa_pairs.append({"question": question, "answer": answer, "verification": verification["verification"]})

    # Step 5: Synthesize

    final_response = self.synthesie_final_response(query, qa_pairs)

    end_time = time.time()


    return {
        "response": final_response,
        "reasoning_steps": {
            "analysis": analysis["analysis"],
            "sub_questions": sub_questions,
            "qa_pairs": qa_pairs
        },
        "processing_time": end_time - start_time
        }

In [49]:
user_prompt ="what is the process of buying a home in the US?"

In [48]:
model = "gpt-4o"
base_llm = BaseLLM(model)
reasoning_llm = ReasoningLLM(base_llm)

print("\n" + "="*50)
print("REASONING LLM:",model)
print("="*50)
reasoning_result = reasoning_llm.reason(user_prompt)
print(f"Final Response:")
print(reasoning_result['response'])
print(f"Sub-questions explored: {len(reasoning_result['reasoning_steps']['sub_questions'])}")
print(f"Time: {reasoning_result['processing_time']:.2f}s")


REASONING LLM: gpt-4o

 ReasoningLLM: Processing 'what is the process of buying a home in the US?
Reasoning LLM: Step 1 - Analzying query..
BaseLLM: making single API call...
Reasoning LLM: Step 2- Decomposing problem...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 1. What steps should I take before starting the ho...
BaseLLM: making single API call...
ReasoningLLM: Step 4- Verifying response...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 2. How do I explore financing options and navigate...
BaseLLM: making single API call...
ReasoningLLM: Step 4- Verifying response...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 3. What should I consider during the home search a...
BaseLLM: making single API call...
ReasoningLLM: Step 4- Verifying response...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 4. What are the important steps in closing the dea...
BaseLLM: making single API call...
ReasoningL